In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import h5py
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
import pywt
from plotly import tools
from plotly.offline import init_notebook_mode, iplot, plot
# import cufflinks as cf
import xgboost as xgb
import plotly.graph_objs as go
from datetime import datetime
from datetime import timedelta
import warnings
import time
from collections import Counter
import scipy
import pywt
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelBinarizer

In [2]:
X_train = h5py.File("original_data/X_train.h5", "r")
y_train_ = pd.read_csv("original_data/y_train.csv").as_matrix()[:, 1].squeeze()
df_train = pd.DataFrame(data=X_train["features"][:])

In [12]:
df_train_final = pd.read_hdf('df_train_final.h5', 'df_train_final')

In [4]:
predictors= list(df_train_final.columns[0:17])+list(df_train_final.columns[81:83]) + list(df_train_final.columns[85:181])

In [21]:
df_train_final1 = df_train_final[predictors]
df_train_final1[var_to_pred] = df_train_final[var_to_pred]

## Binary Labels

In [22]:
var_to_pred = 'SO'

In [23]:
lb = LabelBinarizer()

In [24]:
df_binary = df_train_final1.join(pd.DataFrame(lb.fit_transform(df_train_final1[var_to_pred]),
                          columns=lb.classes_, 
                          index=df_train_final1.index))

In [30]:
df_binary_0 = df_binary.drop(columns=[var_to_pred, 1, 2])
df_binary_0 = df_binary_0.rename(columns={0:var_to_pred})

In [31]:
df_binary_1 = df_binary.drop(columns=[var_to_pred, 0, 2])
df_binary_1 = df_binary_1.rename(columns={1:var_to_pred})

In [32]:
df_binary_2 = df_binary.drop(columns=[var_to_pred, 0, 1])
df_binary_2 = df_binary_2.rename(columns={2:var_to_pred})

In [34]:
df_binary_0.columns

Index(['num_pso', 'mean_amp_pso', 'mean_dur_pso', 'amp_cso', 'dur_cso',
       'time_since_sleep', 'time_in_ds', 'time_in_ls', 'time_in_rs',
       'time_in_ws',
       ...
       'wawelets_87', 'wawelets_88', 'wawelets_89', 'wawelets_90',
       'wawelets_91', 'wawelets_92', 'wawelets_93', 'wawelets_94',
       'wawelets_95', 'SO'],
      dtype='object', length=116)

In [35]:
df_binary_0.to_hdf('df_binary_0.h5', key='df_binary_0')

In [36]:
df_binary_1.to_hdf('df_binary_1.h5', key='df_binary_1')

In [37]:
df_binary_2.to_hdf('df_binary_2.h5', key='df_binary_2')

In [25]:
df_binary_0.to_csv('df_binary_0.csv')

In [34]:
i=0
for row in df_binary_0.iterrows():
    print(row[1][-1])
    i+=1
    if i==4:
        break

1.0
0.0
0.0
0.0


In [39]:
df_binary_0.head()

,num_pso,mean_amp_pso,mean_dur_pso,amp_cso,dur_cso,time_since_sleep,time_in_ds,time_in_ls,time_in_rs,time_in_ws,...,wawelets_2_sec_51,wawelets_2_sec_52,wawelets_2_sec_53,wawelets_2_sec_54,wawelets_2_sec_55,wawelets_2_sec_56,wawelets_2_sec_57,wawelets_2_sec_58,wawelets_2_sec_59,SO
0,237.0,152.658761,341.523207,128.017491,429.0,11379.0,2730.0,3780.0,0.0,480.0,...,-1.269829,-0.505228,0.505317,1.257482,-0.019131,0.001756,0.841372,0.707907,0.660160,1
1,176.0,146.883435,338.039773,119.130849,196.0,6721.0,2580.0,2100.0,0.0,480.0,...,-1.251475,-0.580705,0.453012,1.541071,-0.058753,-0.000819,0.852977,0.727571,0.658971,0
2,456.0,152.376541,335.629386,164.292580,417.0,26832.0,3240.0,7440.0,2130.0,750.0,...,-1.546928,-0.654804,0.599300,1.511609,0.086784,0.006449,0.988418,0.976971,0.774067,0
3,21.0,139.720772,336.285714,159.237082,407.0,1289.0,0.0,60.0,0.0,450.0,...,-1.446389,-0.538793,0.524387,1.378819,-0.017170,-0.005554,0.886979,0.786731,0.691827,0
4,72.0,140.649432,349.875000,130.184278,297.0,2262.0,630.0,960.0,0.0,450.0,...,-1.522254,-0.573445,0.530961,1.567218,0.034848,0.005318,0.946961,0.896734,0.718545,0


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df_binary_0.loc[:, df_binary_0.columns != var_to_pred], df_binary_0[var_to_pred], test_size=0.10,random_state=0, stratify=df_binary_0[var_to_pred])

In [ ]:
y_train = pd.DataFrame(y_train)
y_val = pd.DataFrame(y_val)

## Get binary dataframe columns

In [60]:
df_binary = pd.read_hdf('df_binary_0.h5', 'df_binary_0.h5'.replace('.h5', ''))
predictors = list(list(df_binary.columns[0:81])+list(df_binary.columns[85:180]))
predictors.append(str(df_binary_0.columns[-1]))
print(predictors)
#df_binary = df_binary[predictors]

['num_pso', 'mean_amp_pso', 'mean_dur_pso', 'amp_cso', 'dur_cso', 'time_since_sleep', 'time_in_ds', 'time_in_ls', 'time_in_rs', 'time_in_ws', 'mean', 'max', 'min', 'perm_entropy', 'svd_entropy', 'higuchi_fd', 'detrended_fluctuation', 'fft_0', 'fft_1', 'fft_2', 'fft_3', 'fft_4', 'fft_5', 'fft_6', 'fft_7', 'fft_8', 'fft_9', 'fft_10', 'fft_11', 'fft_12', 'fft_13', 'fft_14', 'fft_15', 'fft_16', 'fft_17', 'fft_18', 'fft_19', 'fft_20', 'fft_21', 'fft_22', 'fft_23', 'fft_24', 'fft_25', 'fft_26', 'fft_27', 'fft_28', 'fft_29', 'fft_30', 'fft_31', 'fft_32', 'fft_33', 'fft_34', 'fft_35', 'fft_36', 'fft_37', 'fft_38', 'fft_39', 'fft_40', 'fft_41', 'fft_42', 'fft_43', 'fft_44', 'fft_45', 'fft_46', 'fft_47', 'fft_48', 'fft_49', 'fft_50', 'fft_51', 'fft_52', 'fft_53', 'fft_54', 'fft_55', 'fft_56', 'fft_57', 'fft_58', 'fft_59', 'fft_60', 'fft_61', 'fft_62', 'fft_63', 'wawelets_1', 'wawelets_2', 'wawelets_3', 'wawelets_4', 'wawelets_5', 'wawelets_6', 'wawelets_7', 'wawelets_8', 'wawelets_9', 'wawelets_

In [62]:
list(list(df_binary.columns[81:85])+list(df_binary.columns[85:180]))

['curr_sleep_stage_2.0',
 'curr_sleep_stage_3.0',
 'slow_o',
 'wawelets_0',
 'wawelets_1',
 'wawelets_2',
 'wawelets_3',
 'wawelets_4',
 'wawelets_5',
 'wawelets_6',
 'wawelets_7',
 'wawelets_8',
 'wawelets_9',
 'wawelets_10',
 'wawelets_11',
 'wawelets_12',
 'wawelets_13',
 'wawelets_14',
 'wawelets_15',
 'wawelets_16',
 'wawelets_17',
 'wawelets_18',
 'wawelets_19',
 'wawelets_20',
 'wawelets_21',
 'wawelets_22',
 'wawelets_23',
 'wawelets_24',
 'wawelets_25',
 'wawelets_26',
 'wawelets_27',
 'wawelets_28',
 'wawelets_29',
 'wawelets_30',
 'wawelets_31',
 'wawelets_32',
 'wawelets_33',
 'wawelets_34',
 'wawelets_35',
 'wawelets_36',
 'wawelets_37',
 'wawelets_38',
 'wawelets_39',
 'wawelets_40',
 'wawelets_41',
 'wawelets_42',
 'wawelets_43',
 'wawelets_44',
 'wawelets_45',
 'wawelets_46',
 'wawelets_47',
 'wawelets_48',
 'wawelets_49',
 'wawelets_50',
 'wawelets_51',
 'wawelets_52',
 'wawelets_53',
 'wawelets_54',
 'wawelets_55',
 'wawelets_56',
 'wawelets_57',
 'wawelets_58',
 'waw